In [ ]:
import sys
import os

import gettelemetry as client

import torch
import torch.nn.functional as F
import mss
import cv2
import time
import numpy as np

import pywinctl as gw
import vgamepad as vg
import gymnasium as gym


In [14]:
class WindowCap():
    def __init__(self, window_name):
        self.window_name = window_name
        self.window = gw.getWindowsWithTitle(window_name)
        if not self.window:
            raise Exception(f"Window with name '{window_name}' not found.")
        self.window = self.window[0]
        self.top = self.window.top
        self.left = self.window.left
        self.width = self.window.width
        self.height = self.window.height
        self.monitor = {"top": self.top, "left": self.left, "width": self.width, "height": self.height}
        self.sct = mss.mss()

    def capture(self):
        img = np.array(self.sct.grab(self.monitor))
        img = cv2.resize(img, (124, 124))
        return img

    def __del__(self):
        self.sct.close()


    def show_window(self):
        with mss.mss() as sct:
            monitor = {"top": self.top, "left": self.left, "width": self.width, "height": self.height}
            frame_count = 0
            fps = 0
            start_time = time.time()

            while True:
                # Capture screen
                img = np.array(sct.grab(monitor))
                # resize image
                img = cv2.resize(img, (124, 124))

                # Convert to grayscale (direct from BGRA)
                #img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)

                # Display the image
                cv2.imshow("Screen Capture", img)

                # Calculate FPS every 10 frames
                frame_count += 1
                if frame_count % 10 == 0:
                    end_time = time.time()
                    fps = frame_count / (end_time - start_time)
                    print(f"FPS: {fps:.2f}")
                    frame_count = 0
                    start_time = time.time()

                # Exit loop on 'q' key press
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                # Release resources
        cv2.destroyAllWindows()

In [8]:
# client
cl = client.TMClient()

try:
    print("Client started, waiting for data...")
    while True:
        data = cl.retrieve_data()
        print(data)
# stop if q is pressed
except KeyboardInterrupt:
    pass
finally:
    cl.close()

Expecting 76 bytes per message (19 floats)
Attempting to connect to localhost:9000...
Client started, waiting for data...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...
Attempting to connect to localhost:9000...
Connection refused. Retrying in 1 second...


TimeoutError: Failed to connect to the server

In [ ]:
# setup gamepad
gamepad = vg.VX360Gamepad()

# 1,3 tensor
def send_action(action):
    steering, brake, throttle = action
    gamepad.left_joystick_float(x_value_float=steering, y_value_float=0.0) # left/right
    gamepad.left_trigger_float(value_float=brake) # brake
    gamepad.right_trigger_float(value_float=throttle)
    gamepad.update()


In [ ]:
class TrackmaniaEnv(gym.Env) :
    def __init__(self, window_name):
        self.window = WindowCap(window_name)
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(124, 124, 3), dtype=np.uint8)
        self.reward_range = (-np.inf, np.inf)
        self.metadata = {'render.modes': ['human']}
        self.spec = None
        self.done = False
        self.reward = 0
        self.prev_action = np.array([0, 0, 0])
        self.prev_obs = np.zeros((124, 124, 3), dtype=np.uint8)

    def reset(self):
        return self.window.capture()

    def step(self, action):
        send_action(action)

        self.prev_action = action
        obs = self.window.capture()
        reward = self.reward
        done = self.done
        info = {}
        return obs, reward, done, info


    

    def close(self):
        del self.window

In [15]:
# LSTM, SAC model
# feed in 4 frames at a time
# output is the predicted action
# train on the fly
# save model
# load model


# feature extraction
# 1. CNN
# 2. LSTM
# 3. SAC



